# Workbook
----------------------------------------

## Testing preprocess module
- 


In [1]:
%matplotlib nbagg

import preprocess

pp = preprocess.preprocess()
X, y = pp.getData("../homework2/data/data_small5.in", 10) #path, clip numDivide
# X, y = pp.getData("../serverTest/MusicClassification/data/data.in", 50000) #path, clip length

(X_center, y_center), (X_classifier, y_classifier), (X_test, y_test) = pp.splitData(
    X, y, randState=1985, TestTrainRatio=0.1, ClassifierCenterRatio=0.6)

print X_center.shape, X_classifier.shape, X_test.shape

X_center_features = pp.featureExtraction(X_center) #includes MFCC and normalization
print "X_center_features.shape", X_center_features.shape #Note: this just flatten everything


/Users/apple/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
/Users/apple/anaconda/lib/python2.7/site-packages/librosa/core/audio.py:37: UserWarning: Could not import scikits.samplerate. Falling back to scipy.signal
  warnings.warn('Could not import scikits.samplerate. '


X size/number of songs: 40 Number of clips per song: 100 y size: 40
(14, 100, 5000) (22, 100, 5000) (4, 100, 5000)
--> After MFCC X.shape (14, 100, 12, 10)
--> X_train_flattened.shape (1400, 12, 10)
--> After transpose X_train_flattened.shape (1400, 10, 12)
X_center_features.shape (14000, 12)


## Find Center
- We need to play with K and maybe batch_size

In [2]:
from sklearn.cluster import MiniBatchKMeans
print "--------------", "START MiniBatchKMeans", "--------------"
n, dim = X_center_features.shape
clf = MiniBatchKMeans(n_clusters=10, batch_size=int(n*0.1), max_iter=200).fit(X_center_features) #X : array-like, shape = [n_samples, n_features]
centroids = clf.cluster_centers_
print "centroids.shape", centroids.shape

-------------- START MiniBatchKMeans --------------
centroids.shape (10, 12)


## VLAD

In [3]:
import vlad

trainX_centerSubstract, trainy_centerSubstract = vlad.my_vlad(centroids, X_classifier, y_classifier)
testX_centerSubstract, testy_centerSubstract = vlad.my_vlad(centroids, X_test, y_test)

print trainX_centerSubstract.shape
print trainy_centerSubstract.shape

vlad before X.shape (22, 100, 5000)
vlad after X.shape (22, 100, 12, 10)
After transpose Xt.shape (22, 100, 10, 12)
vlad before X.shape (4, 100, 5000)
vlad after X.shape (4, 100, 12, 10)
After transpose Xt.shape (4, 100, 10, 12)
(22, 12)
(22,)


In [4]:
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
import numpy as np
param_grid = {'n_neighbors': np.arange(3, 8),'weights':('uniform', 'distance'), 'algorithm':('auto','ball_tree', 'kd_tree', 'brute') }
np.set_printoptions(suppress=True)
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, verbose=3)
grid_search.fit(trainX_centerSubstract, trainy_centerSubstract)
print "----------------", "Done grid search", "----------------"

Fitting 3 folds for each of 40 candidates, totalling 120 fits
[CV] n_neighbors=3, weights=uniform, algorithm=auto ..................
[CV]  n_neighbors=3, weights=uniform, algorithm=auto, score=0.272727 -   0.0s
[CV] n_neighbors=3, weights=uniform, algorithm=auto ..................
[CV]  n_neighbors=3, weights=uniform, algorithm=auto, score=0.285714 -   0.0s
[CV] n_neighbors=3, weights=uniform, algorithm=auto ..................
[CV]  n_neighbors=3, weights=uniform, algorithm=auto, score=0.250000 -   0.0s
[CV] n_neighbors=3, weights=distance, algorithm=auto .................
[CV]  n_neighbors=3, weights=distance, algorithm=auto, score=0.181818 -   0.0s
[CV] n_neighbors=3, weights=distance, algorithm=auto .................
[CV]  n_neighbors=3, weights=distance, algorithm=auto, score=0.142857 -   0.0s
[CV] n_neighbors=3, weights=distance, algorithm=auto .................
[CV]  n_neighbors=3, weights=distance, algorithm=auto, score=0.500000 -   0.0s
[CV] n_neighbors=4, weights=uniform, algo

/Users/apple/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)
[Parallel(n_jobs=1)]: Done  31 tasks       | elapsed:    0.1s



[CV] n_neighbors=6, weights=uniform, algorithm=auto ..................
[CV]  n_neighbors=6, weights=uniform, algorithm=auto, score=0.272727 -   0.0s
[CV] n_neighbors=6, weights=uniform, algorithm=auto ..................
[CV]  n_neighbors=6, weights=uniform, algorithm=auto, score=0.285714 -   0.0s
[CV] n_neighbors=6, weights=uniform, algorithm=auto ..................
[CV]  n_neighbors=6, weights=uniform, algorithm=auto, score=0.250000 -   0.0s
[CV] n_neighbors=6, weights=distance, algorithm=auto .................
[CV]  n_neighbors=6, weights=distance, algorithm=auto, score=0.181818 -   0.0s
[CV] n_neighbors=6, weights=distance, algorithm=auto .................
[CV]  n_neighbors=6, weights=distance, algorithm=auto, score=0.285714 -   0.0s
[CV] n_neighbors=6, weights=distance, algorithm=auto .................
[CV]  n_neighbors=6, weights=distance, algorithm=auto, score=0.500000 -   0.0s
[CV] n_neighbors=7, weights=uniform, algorithm=auto ..................
[CV]  n_neighbors=7, weights=un

[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:    0.3s finished


In [5]:
predict = grid_search.predict(testX_centerSubstract)
print grid_search.score(testX_centerSubstract, testy_centerSubstract)
print grid_search.best_params_

0.25
{'n_neighbors': 4, 'weights': 'uniform', 'algorithm': 'auto'}
